<a href="https://colab.research.google.com/github/rootAkash/reinforcement_learning/blob/master/Soft%20AC/sac_latest(2019ver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from collections import deque
import random
from tqdm import tqdm
class Policy(nn.Module):

    def __init__(self,observation_size,action_size,hidden_units):
        super().__init__()
        self.observation_size=observation_size
        self.hidden_units=hidden_units
        self.action_size = action_size
        self.h1 = nn.Linear(self.observation_size, self.hidden_units)  
        self.h2 = nn.Linear(self.hidden_units,self.hidden_units)
        self.h3 = nn.Linear(self.hidden_units,self.hidden_units)
        self.h4 = nn.Linear(self.hidden_units,self.hidden_units)
        self.h5 = nn.Linear(self.hidden_units,self.hidden_units)
        self.h6 = nn.Linear(self.hidden_units,self.hidden_units)
        self.mu = nn.Linear(self.hidden_units,self.action_size)
        self.sigma = nn.Linear(self.hidden_units,self.action_size)
    def forward(self, x):
        x = F.relu(self.h1(x))
        x = F.relu(self.h2(x))
        x = F.relu(self.h3(x))
        x = F.relu(self.h4(x))
        x = F.relu(self.h5(x))
        x = F.relu(self.h6(x))
        mus = torch.tanh(self.mu(x))
        sigs= F.softplus(self.sigma(x))
        sigs= torch.clamp(sigs, min=0.001, max=100)#1e-22 , 1e+02
        return mus , sigs
    def predict(self, x):
        with torch.no_grad():
          output=self.forward(x)
        return output  

class Q_net(nn.Module):

    def __init__(self,observation_size,action_size,hidden_units):
        super().__init__()
        self.observation_size=observation_size
        self.hidden_units=hidden_units
        self.action_size = action_size
        self.h1 = nn.Linear(self.observation_size, self.hidden_units) 
        self.a1 = nn.Linear(self.action_size, self.hidden_units) 
        self.h2 = nn.Linear(self.hidden_units*2,self.hidden_units)
        self.h3 = nn.Linear(self.hidden_units,self.hidden_units)
        self.h4 = nn.Linear(self.hidden_units,self.hidden_units)
        self.h5 = nn.Linear(self.hidden_units,self.hidden_units)
        self.h6 = nn.Linear(self.hidden_units,self.hidden_units)
        self.q = nn.Linear(self.hidden_units,1)
    def forward(self, x,a):
        x = F.relu(self.h1(x))
        a = F.relu(self.a1(a))
        x = torch.cat([x,a], dim=1)
        x = F.relu(self.h2(x))
        x = F.relu(self.h3(x))
        x = F.relu(self.h4(x))
        x = F.relu(self.h5(x))
        x = F.relu(self.h6(x))
        qout = self.q(x)
        return qout
    def predict(self, x,a):
        with torch.no_grad():
          output=self.forward(x,a)
        return output

def remember(s,a,r,ns,d):
  #s=s.ravel()
  #ns=ns.ravel()
  memory.append([s,a,np.array([r]),ns,np.array([d])])
def sample_games(buffer,batch_size):
  # Sample game from buffer either uniformly or according to some priority
  #print("samplig from .",len(buffer))
  return list(np.random.choice(len(buffer),batch_size))
def soft_update(target, source, tau):
  for target_param, param in zip(target.parameters(), source.parameters()):
    target_param.data.copy_(target_param.data * (1.0 - tau) + param.data * tau)
  return target
def get_log_pdf_multi(x,mean,std):
  k= x.shape[1] #action dim
  pi = 3.1415926
  cov = std**2
  det = torch.prod(cov,dim=1,keepdim=True)
  norm_const = 1.0/ ( np.power(2*pi,k/2) * torch.pow(det,0.5) )
  prod  = (1/cov)*torch.square(x - mean)
  prod2 =torch.sum(prod,dim=1,keepdim=True) 
  pdf = norm_const * torch.exp( -0.5 *prod2)
  final_log_pdf = torch.log(pdf+1e-07)
  return final_log_pdf

def get_entropy_multi(x, mean, std,Act):
  #log pdf (squashed guassian) = log pdfguassian(mupolicy,sigma_policy) - sum of  log(1-A**2) ; where each A is component of tanh squahed action vector 
  log_pdf_final = get_log_pdf_multi(x,mean,std) - torch.sum(torch.log(1- torch.square(Act) +1e-07),dim=1,keepdim=True)
  return -log_pdf_final



def replay_and_train(target_entropy,log_alpha,policy,q_1,q_2,t_q_1,t_q_2,popt,qopt,log_alpha_opt,size=128):
  #target entropy is a constant so no need to make it into torch tensor
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  gamma=0.99
  mse = nn.MSELoss()
  alpha = np.exp(log_alpha.item())#get new alpha value
  sample_size=size
  if len(memory) < sample_size:
    return
  samples=random.sample(memory,sample_size)
  s,a,r,ns,d=zip(*samples)

  #s,a,r,ns,d = list(s),list(a),list(r),list(ns),list(d)
  #print(s,a,r,ns,d)
  s= torch.tensor(s).float().to(device)
  a= torch.tensor(a).float().to(device)
  r= torch.tensor(r).float().to(device)
  ns= torch.tensor(ns).float().to(device)
  d= torch.tensor(d).float().to(device)

  new_n_mu,new_n_sig = policy.predict(ns)
  n_E =np.random.multivariate_normal(np.zeros_like(new_n_mu[0].numpy()),np.diag(np.ones_like(new_n_mu[0].numpy())),size) 
  n_E = torch.tensor(n_E).float().to(device)
  n_Act_guassian = new_n_mu + n_E*new_n_sig # guassian action using reparametrisation
  new_next_Action  =torch.tanh(n_Act_guassian)  #final squashed guassian action
  n_entropy = get_entropy_multi(n_Act_guassian,new_n_mu,new_n_sig,new_next_Action)
  yq = r + gamma*(1-d)*torch.minimum(t_q_1.predict(ns,new_next_Action),t_q_2.predict(ns,new_next_Action)) + alpha*n_entropy # value target should be bootstrapped to current policy
  #training nets 
  #train q nets
  qloss = mse(q_1(s,a),yq) + mse(q_2(s,a),yq)
  qopt.zero_grad()                                                                                                          #    
  qloss.backward()                                                                                                         #
  qopt.step() 
  #train policy
  new_mu,new_sig = policy(s)
  E =np.random.multivariate_normal(np.zeros_like(new_mu[0].detach().numpy()),np.diag(np.ones_like(new_mu[0].detach().numpy())),size) 
  E = torch.tensor(E).float().to(device)
  Act_guassian = new_mu + E*new_sig # guassian action using reparametrisation
  new_Action  =torch.tanh(Act_guassian)  #final squashed guassian action
  entropy = get_entropy_multi(Act_guassian,new_mu,new_sig,new_Action)

  policy_objective = torch.minimum(q_1(s,new_Action),q_2(s,new_Action))#state from buffer action from recent policy and not from buffer to train to maximise q
  final_policy_objective = policy_objective + alpha*entropy # maximise this
  final_policy_loss = - torch.mean(final_policy_objective)  #therefore minimise this
  popt.zero_grad()                                                                                                          #    
  final_policy_loss.backward()                                                                                                         #
  popt.step() 
  #train alpha
  log_alpha_loss = torch.mean(log_alpha*(entropy.detach() - target_entropy))
  log_alpha_opt.zero_grad()                                                                                                          #    
  log_alpha_loss.backward()                                                                                                         #
  log_alpha_opt.step() 
  #train t_value
  soft_update(target=t_q_1, source=q_1, tau=0.001)
  soft_update(target=t_q_2, source=q_2, tau=0.001)
  return qloss.item(),final_policy_loss.item(),log_alpha_loss.item(),torch.mean(entropy).item()


  



In [ ]:

import gym

memory=deque(maxlen=5000000)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#env,networks ad optimisers
env=gym.make('Pendulum-v0')
#env=gym.make('MountainCarContinuous-v0')
#env=gym.make('LunarLanderContinuous-v2')
env=env.unwrapped

s_dim = env.observation_space.shape[0]
print(s_dim)
a_dim = env.action_space.shape[0]
print(a_dim)
a_bound = env.action_space.high[0]
print(a_bound)
#########################################################################
target_entropy = -a_dim# -dim(A)
log_alpha = torch.tensor([0.0], requires_grad=True).to(device)
policy = Policy(s_dim,a_dim,100).to(device)
q_1 = Q_net(s_dim,a_dim,100).to(device)
q_2 = Q_net(s_dim,a_dim,100).to(device)
t_q_1 = Q_net(s_dim,a_dim,100).to(device)
t_q_2 = Q_net(s_dim,a_dim,100).to(device)
soft_update(target=t_q_1, source=q_1, tau=1)
soft_update(target=t_q_2, source=q_2, tau=1)
qopt = optim.Adam(list(q_1.parameters()) + list(q_2.parameters()),lr=0.001)
popt = optim.Adam(policy.parameters(),lr=0.001)      
log_alpha_opt =  optim.Adam(params=[log_alpha],lr=0.001)
#########################################################################
max_steps=5000
train_iter = 2000  # 1000, batch of 8/16 works best for now
test_eps = 335#after this no training
warmup = 20 # before this only buffer is filled without training
episodes = test_eps+200
for e in range(episodes):
  done = False
  s = env.reset()
  rew = 0 
  stp=0
  while not done:
    new_mu,new_sig = policy.predict(torch.tensor([s]).float().to(device))
    new_mu,new_sig=new_mu.cpu().numpy(),new_sig.cpu().numpy()
    #print(new_mu,new_sig,new_mu[0],new_sig[0])
    E =np.random.multivariate_normal(np.zeros_like(new_mu[0]),np.diag(np.ones_like(new_mu[0])))#np.random.normal(mu=0,std_dev=1) diag cov of 1 is same as std dev of 1
    Act_guassian = new_mu[0] + new_sig[0]*E # guassian action using reparametrisation
    act  =np.tanh(Act_guassian)
    if e>test_eps-10:
      env.render()
      #act = new_mu[0] # testing

    s_,r,done,_=env.step(act*a_bound)
    #if done:
    #  r = 10000
    if stp>max_steps:
      done = True
    if e<=test_eps:    
      remember(s,act,r,s_,done)
    s=s_
    rew+=r
    stp+=1
  print(e,rew)
  if e>warmup and e <= test_eps:
    print("training")
    p=0
    q=0
    a=0
    e=0
    for i in  tqdm(range(train_iter)):
      ql,pl,al,ee=replay_and_train(target_entropy,log_alpha,policy,q_1,q_2,t_q_1,t_q_2,popt,qopt,log_alpha_opt,size=8)
      p+=pl
      q+=ql
      a+=al
      e+=ee
    print("p loss",p/train_iter,"|q loss ",q/train_iter,"|alpha loss ",a/train_iter,"|avg entropy ",e/train_iter)  
  

3
1
2.0
0 -25222.842255454816
1 -30296.520427303873
2 -29253.245579475373
3 -26590.32169724874
4 -29588.22246068339
5 -28997.538767979346
6 -30976.4101496262
7 -30611.770735596507
8 -35826.86704074124
9 -27615.34144057221
10 -34340.218415652686
11 -28772.62722366432
12 -38849.47271567901
13 -27188.860613252553
14 -33812.262474530355
15 -32594.59622314251
16 -31784.288941319403
17 -28061.169291971408
18 -29517.562886011136
19 -28322.741513295015
20 -33584.5164193784


  0%|          | 6/2000 [00:00<00:33, 59.13it/s]

21 -31818.416831428232
training


100%|██████████| 2000/2000 [00:34<00:00, 58.07it/s]


p loss 49.090103847935794 |q loss  10.571634257160127 |alpha loss  -0.8737437579840771 |avg entropy  0.1898439302444458


...

p loss 117.1976464176178 |q loss  63.31087055319548 |alpha loss  -0.14882694161869586 |avg entropy  -0.9403860767222941


  0%|          | 6/2000 [00:00<00:38, 51.40it/s]

34 -378.7351438432642
training


 26%|██▋       | 528/2000 [00:09<00:26, 55.15it/s]


KeyboardInterrupt: ignored

In [ ]:

import gym

memory=deque(maxlen=5000000)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#env,networks ad optimisers
#env=gym.make('Pendulum-v0')
env=gym.make('MountainCarContinuous-v0')
#env=gym.make('LunarLanderContinuous-v2')
env=env.unwrapped

s_dim = env.observation_space.shape[0]
print(s_dim)
a_dim = env.action_space.shape[0]
print(a_dim)
a_bound = env.action_space.high[0]
print(a_bound)
#########################################################################
target_entropy = -a_dim# -dim(A)
log_alpha = torch.tensor([0.0], requires_grad=True).to(device)
policy = Policy(s_dim,a_dim,256).to(device)
q_1 = Q_net(s_dim,a_dim,256).to(device)
q_2 = Q_net(s_dim,a_dim,256).to(device)
t_q_1 = Q_net(s_dim,a_dim,256).to(device)
t_q_2 = Q_net(s_dim,a_dim,256).to(device)
soft_update(target=t_q_1, source=q_1, tau=1)
soft_update(target=t_q_2, source=q_2, tau=1)
qopt = optim.Adam(list(q_1.parameters()) + list(q_2.parameters()),lr=0.001)
popt = optim.Adam(policy.parameters(),lr=0.001)      
log_alpha_opt =  optim.Adam(params=[log_alpha],lr=0.001)
#########################################################################
max_steps=5000
train_iter = 1000  # 1000, batch of 8/16 works best for now
test_eps = 335#after this no training
warmup = 20 # before this only buffer is filled without training
episodes = test_eps+200
for e in range(episodes):
  done = False
  s = env.reset()
  rew = 0 
  stp=0
  for step in range(max_steps):
    if done:
      s = env.reset()    
    new_mu,new_sig = policy.predict(torch.tensor([s]).float().to(device))
    new_mu,new_sig=new_mu.cpu().numpy(),new_sig.cpu().numpy()
    #print(new_mu,new_sig,new_mu[0],new_sig[0])
    E =np.random.multivariate_normal(np.zeros_like(new_mu[0]),np.diag(np.ones_like(new_mu[0])))#np.random.normal(mu=0,std_dev=1) diag cov of 1 is same as std dev of 1
    Act_guassian = new_mu[0] + new_sig[0]*E # guassian action using reparametrisation
    act  =np.tanh(Act_guassian)
    if e>test_eps-10:
      env.render()
      #act = new_mu[0] # testing
    if e < warmup :
      # for some additional exploration not necesserily needed
      if E > 0.5:
        act = np.clip(a_bound*E,-a_bound,a_bound)
    s_,r,done,_=env.step(act*a_bound)
    if done :
      r =r+1000 # to encourage reaching target more
      print("reached")

    if e<=test_eps:    
      remember(s,act,r,s_,done)
    s=s_
    rew+=r
    stp+=1
  print(e,rew)
  if e>0 and e <= test_eps:
    print("training")
    p=0
    q=0
    a=0
    e=0
    for i in  range(train_iter):
      ql,pl,al,ee=replay_and_train(target_entropy,log_alpha,policy,q_1,q_2,t_q_1,t_q_2,popt,qopt,log_alpha_opt,size=16)
      p+=pl
      q+=ql
      a+=al
      e+=ee
      if i % (train_iter//10)==0:
        print('.',end='')#tqdm sometimes has issues in colab so did this 
    print('|')        
    print("p loss",p/train_iter,"|q loss ",q/train_iter,"|alpha loss ",a/train_iter,"|avg entropy ",e/train_iter)  
  

2
1
1.0
reached
0 913.99809764594
reached
1 916.1790499573405
training
..........|
p loss -0.9757406290769577 |q loss  302.5312634554472 |alpha loss  -0.8413645657507004 |avg entropy  0.6812703840136528
2 -199.42869662691916
training
..........|
p loss -0.6142752554416656 |q loss  151.1950949425447 |alpha loss  -2.510945767402649 |avg entropy  0.675104011297226
3 -194.32615789042165
training
..........|
p loss -0.494650959700346 |q loss  302.3434837314311 |alpha loss  -4.109535607814789 |avg entropy  0.6494722853600979
4 -174.4450626007392
training
..........|
p loss -0.42509693949669597 |q loss  302.2682649302877 |alpha loss  -5.422665097236633 |avg entropy  0.5657113275080919
5 -148.54560327173942
training
..........|
p loss -0.20876643967628478 |q loss  0.001610347646928858 |alpha loss  -4.468893990039826 |avg entropy  0.04021861305925995
6 -133.44223360142158
training
..........|
p loss -0.17914595772325992 |q loss  9.0638146870333e-05 |alpha loss  -3.5103050490617753 |avg entropy 